In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [26]:
import struct
import os

def read_mnist_images(file_path):
    with open(file_path, 'rb') as f:
        magic, num_images = struct.unpack('>II', f.read(8))
        rows, cols = struct.unpack('>II', f.read(8))
        images = np.fromfile(f, dtype=np.uint8).reshape(num_images, rows * cols)
    return images

def read_mnist_labels(file_path):
    with open(file_path, 'rb') as f:
        magic, num_labels = struct.unpack('>II', f.read(8))
        labels = np.fromfile(f, dtype=np.uint8)
    return labels

# Replace these paths with your actual file paths
train_images_path = 'Dataset/train-images-idx3-ubyte/train-images-idx3-ubyte'  # Update with your path
train_labels_path = 'Dataset/train-labels-idx1-ubyte/train-labels-idx1-ubyte'  # Update with your path
test_images_path = 't10k-images-idx3-ubyte'    # Update with your path
test_labels_path = 't10k-labels-idx1-ubyte'    # Update with your path

# Load data
train_images = read_mnist_images(train_images_path)
train_labels = read_mnist_labels(train_labels_path)

# Create DataFrames
# First for the features (images)
train_df = pd.DataFrame(train_images)
# Add the labels
train_df['label'] = train_labels


In [24]:
df=train_df.copy()

In [38]:
cols = df.columns.tolist()
new_order = [cols[-1]] + cols[:-1]
df = df[new_order]

In [39]:
df.head()

,label,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# We need our data in a numpy array for the model and manipulation
data=np.array(df)

In [50]:
# Split data into train and validation
# m is row and n is column
m,n=data.shape
np.random.shuffle(data)
# Validation set is the first 1000 samples
data_dev=data[0:1000].T
Y_dev=data_dev[0]
X_dev=data_dev[1:n]
# Training set is the rest of the samples
data_train=data[1000:m].T
Y_train=data_train[0]
X_train=data_train[1:n]


In [60]:
def init_param():
    W1=np.random.rand(10,784)-0.5
    b1=np.random.rand(10,1)-0.5
    W2=np.random.rand(10,10)-0.5
    b2=np.random.rand(10,1)-0.5
    return W1,b1,W2,b2

In [64]:
def RelU(Z):
    if(Z>0):
        return Z
    else:
        return 0

In [65]:
def SoftMax(Z):
    return np.exp(Z)/ np.sum(np.exp(Z))

In [62]:
def forward_propagation(X,W1,b1,W2,b2):
    Z1=W1.dot(X) + b1
    A1= RelU(Z1)
    Z2=W2.dot(A1)+b2
    A2=SoftMax(Z2)
    return A1,A2

In [ ]:
def back_propagation(Z1,A1,Z2,A2,W2,Y):
    